In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

jar_files = [
    "/util/clickhouse-jdbc-0.3.2-all.jar"
]

# Initialize Spark session with JARs
spark = SparkSession.builder \
    .appName("combina_shapes") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", ",".join(jar_files)) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key","NzUmXT2E7YJIY3m9yFBz") \
    .config("spark.hadoop.fs.s3a.secret.key","5SCMGEtt52EziOcTUi9dyMt5dfFE2MBaK1ZaskPX") \
    .config("spark.hadoop.fs.s3a.endpoint","http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()


url = "jdbc:ch://clickhouse:8123/cameraaberta"
user = "admin" 
password = "admin"  
driver = "com.clickhouse.jdbc.ClickHouseDriver"

display(spark)

In [2]:
# pip install geopy

In [3]:
import geopy.distance


In [4]:
df_stops = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/stops.txt").toPandas()
# show data
df_stops.head(10)

# df_stops.printSchema()

,stop_id,stop_name,stop_desc,stop_lat,stop_lon
0,18848,Clínicas,None,-23.554022,-46.671108
1,18849,Vila Madalena,None,-23.546498,-46.691141
2,18850,Consolação,None,-23.558094,-46.660205
3,18851,Conceição,None,-23.635039,-46.641239
4,18852,Jabaquara,None,-23.646033,-46.641028
5,18853,São Judas,None,-23.625882,-46.640936
6,18854,Saúde,None,-23.618245,-46.639139
7,18855,Praça Da Árvore,None,-23.610583,-46.637918
8,18856,Santa Cruz,None,-23.598541,-46.636638
9,18857,Vila Mariana,None,-23.589359,-46.634677


In [5]:
df_shapes = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/shapes.txt").toPandas()
# show data
df_shapes.head(10)

# df_shapes.printSchema()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,81072,-23.432024,-46.787121,1,0.000000
1,81072,-23.431827,-46.787153,2,22.061337
2,81072,-23.431764,-46.787163,3,29.113285
3,81072,-23.431614,-46.787187,4,45.906006
4,81072,-23.431590,-46.787191,5,48.596806
5,81072,-23.431554,-46.787196,6,52.615639
6,81072,-23.431512,-46.787202,7,57.306896
7,81072,-23.431438,-46.787210,8,65.543129
8,81072,-23.431388,-46.787215,9,71.103470
9,81072,-23.431378,-46.787218,10,72.248848


In [6]:
df_trips = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/trips.txt").toPandas()
# show data
df_trips.head(10)

# df_shapes.printSchema()

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id
0,1012-10,USD,1012-10-0,Jd. Monte Belo,0,81072
1,1012-10,USD,1012-10-1,Term. Jd. Britania,1,81073
2,1012-21,U__,1012-21-0,Jd. Rosinha,0,81195
3,1014-10,U__,1014-10-0,Cptm Perus,0,82855
4,1015-10,USD,1015-10-0,Chác. Maria Trindade,0,81148
5,1016-10,USD,1016-10-0,Shop. Center Norte,0,72283
6,1016-10,USD,1016-10-1,Cem. Do Horto,1,82172
7,1017-10,USD,1017-10-0,Conexão Vl. Iório,0,72355
8,1017-10,USD,1017-10-1,Perus,1,72356
9,1018-10,USD,1018-10-0,Metrô Santana,0,71029


In [9]:
df_stop_times = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/stop_times.txt").toPandas()
# show data
df_stop_times.head(10)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,1012-10-0,2025-02-04 18:00:00,2025-02-04 18:00:00,301790,1
1,1012-10-0,2025-02-04 18:01:35,2025-02-04 18:01:35,301764,2
2,1012-10-0,2025-02-04 18:03:10,2025-02-04 18:03:10,301724,3
3,1012-10-0,2025-02-04 18:04:45,2025-02-04 18:04:45,301730,4
4,1012-10-0,2025-02-04 18:06:20,2025-02-04 18:06:20,30003042,5
5,1012-10-0,2025-02-04 18:07:55,2025-02-04 18:07:55,30003045,6
6,1012-10-0,2025-02-04 18:09:30,2025-02-04 18:09:30,30003044,7
7,1012-10-0,2025-02-04 18:11:05,2025-02-04 18:11:05,30003043,8
8,1012-10-0,2025-02-04 18:12:40,2025-02-04 18:12:40,30003013,9
9,1012-10-0,2025-02-04 18:14:15,2025-02-04 18:14:15,301693,10


In [10]:
def get_shape_from_id(shape_id):
    return df_shapes[df_shapes['shape_id'] == shape_id][['shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence']].values

get_shape_from_id(62557)

array([[-2.3654077e+01, -4.6762517e+01,  1.0000000e+00],
       [-2.3653972e+01, -4.6762661e+01,  2.0000000e+00],
       [-2.3653877e+01, -4.6762792e+01,  3.0000000e+00],
       ...,
       [-2.3654177e+01, -4.6762379e+01,  1.0790000e+03],
       [-2.3654077e+01, -4.6762517e+01,  1.0800000e+03],
       [-2.3653972e+01, -4.6762661e+01,  1.0810000e+03]])

In [15]:
def get_trips_stop_seq(trip_id):
    trip_seq = df_stop_times[df_stop_times['trip_id'] == trip_id][['stop_id','stop_sequence']].values
    result = [ (i,tuple(df_stops[df_stops['stop_id'] == s].values[0])) for s,i in trip_seq ]
    return result
# get_trips_stop_seq('1012-10-0')

In [16]:
def find_closest(list_pos, s_lat, s_long):
    # print(s_lat, s_long)
    best_pos_index = None
    best_dist = 999999
    for lat, long, shape_pt_sequence in list_pos:
        # print(lat, long, shape_pt_sequence)
        dist = geopy.distance.geodesic((s_lat, s_long), (lat, long)).meters
        if dist < best_dist:
            best_dist = dist
            best_pos_index = shape_pt_sequence
    return best_pos_index, best_dist

In [17]:
def conciliar_trip_stop(trip_id, shape_id):
    trip_seq = get_trips_stop_seq(trip_id)
    shape = get_shape_from_id(shape_id)
    index = {}
    for i, stop in trip_seq:
        closest = find_closest(shape, stop[3], stop[4])
        # print(closest) # shape_pt_sequence, distance
        index[closest[0]] = (stop[0], closest[1])

    shape_for_trip = df_shapes[df_shapes['shape_id'] == shape_id].copy() 
    shape_for_trip['stop_id'] = [index.get(x, (None, None))[0] for x in shape_for_trip['shape_pt_sequence'].values]
    shape_for_trip['stop_distance'] = [index.get(x, (None, None))[1] for x in shape_for_trip['shape_pt_sequence'].values]
    return shape_for_trip
                     
trip = conciliar_trip_stop('1012-10-0', 81072)

In [18]:
trip

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,stop_id,stop_distance
0,81072,-23.432024,-46.787121,1,0.000000,NaN,NaN
1,81072,-23.431827,-46.787153,2,22.061337,301790.0,10.975496
2,81072,-23.431764,-46.787163,3,29.113285,NaN,NaN
3,81072,-23.431614,-46.787187,4,45.906006,NaN,NaN
4,81072,-23.431590,-46.787191,5,48.596806,NaN,NaN
...,...,...,...,...,...,...,...
290,81072,-23.444853,-46.799501,291,6890.008800,NaN,NaN
291,81072,-23.444664,-46.799499,292,6910.941400,NaN,NaN
292,81072,-23.444473,-46.799500,293,6939.022900,NaN,NaN
293,81072,-23.444412,-46.799515,294,6946.085900,301771.0,4.520668
